In [22]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import words as english_words
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import fasttext
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [23]:
df=pd.read_csv('./kabitakitchen.csv',encoding='Latin-1')

df2=pd.read_csv('./Nishafin.csv')

In [24]:
df=df.drop('id',axis=1)
df2=df2.drop('id',axis=1)

In [25]:
new_df=pd.concat([df,df2],ignore_index=True)

In [26]:
new_df.to_csv('combined_dataset.csv', index=False)

In [27]:
custom_hindi_stops=[
    "aadi", "aaj", "aap", "aapn", "aata", "aati", "aaya", "aay", "ab", "abb",
    "abbey", "abe", "abhi", "accha", "acha", "achcha", "agar", "aint", "aisa", "ais",
    "aisi", "alag", "andar", "ap", "apan", "apna", "apnaa", "apn", "apni", "aur", "avum",
    "aya", "aye","b", "baad", "baar", "bad", "bahut", "bana", "banai", "banao", "banaya",
    "banay", "banayi", "banda", "band", "bandi", "bane", "bani", "ba", "bata", "batao",
    "bhai", "bheetar", "bhi", "bhitar", "bht", "bilkul", "bohot", "bol", "bola", "bole",
    "boli", "bolo", "bolta", "bolt", "bolti", "bro", "btw", "caus", "chahiy", "chaiy",
    "chal", "chalega", "chhaiy", "clearli", "com", "de", "dede", "dega", "degi", "dekh",
    "dekha", "dekh", "dekhi", "dekho", "deng", "dhang", "di", "dijiy", "diya", "diyaa",
    "diy", "diyo", "doosra", "doosr", "dunga", "dungi", "dusra", "dusr", "dusri", "dvaara",
    "dvara", "dwaara", "dwara", "edu", "eg", "ek", "etc", "fr", "gaya", "gay", "gayi",
    "ghar", "h" ,"haan","hi" "hai", "hain", "hamara", "hamar", "hamari", "hamn", "han", "har",
    "hardli","hoga", "hoge", "hogi", "hona", "honaa", "hone", "hong", "hongi", "honi",
    "hota", "hotaa", "hote", "hoti", "hoyeng", "hoyengi", "hu", "hua", "hue", "huh",
    "hui", "hum", "humein", "humn", "hun", "huy", "huyi", "i'm","ie", "imo", "inasmuch", "inc",
    "inh", "inhi", "inho", "inka", "inkaa", "ink", "inki", "inn", "inner", "ins",
    "insofar", "ise", "isi", "iska", "iskaa", "isk", "iski", "ism", "isnt", "iss", "iss",
    "issi", "isski", "itna", "itn", "itni", "itno", "ityaadi", "ityadi","j", "ja", "jaa",
    "jab", "jabh", "jaha", "jahaan", "jahan", "jaisa", "jais", "jaisi", "jata", "jayega",
    "jidhar", "jin", "jinh", "jinhi", "jinho", "jinhon", "jinka", "jink", "jinki", "jinn",
    "ji", "jise", "jiska", "jisk", "jiski", "jism", "jiss", "jiss", "jitna", "jitn",
    "jitni", "jo", "jyaada", "jyada","k","ka", "kaafi", "kab", "kabhi", "kafi", "kaha",
    "kahaa", "kahaan", "kahan", "kahi", "kahin", "kaht", "kaisa", "kais", "kaisi", "kal",
    "kam", "kar", "kara", "kare", "karega", "karegi", "karen", "kareng", "kari", "kark",
    "karna", "karn", "karni", "karo", "karta", "kart", "karti", "karu", "karun", "karunga",
    "karungi", "kaun", "kaunsa", "kayi", "kch", "ke", "keh", "keht", "kept", "khud",
    "ki", "kin", "kine", "kinh", "kinho", "kinka", "kink", "kinki", "kinko", "kinn",
    "kino", "ki", "kise", "kisi", "kiska", "kisk", "kiski", "kisko", "kisliy", "kisn",
    "kitna", "kitn", "kitni", "kitno", "kiya", "kiy", "ko", "koi", "kon", "konsa",
    "koyi", "krna", "krne", "kuch", "kuchch", "kuchh", "kul", "kull", "kya", "kyaa",
    "kyu", "kyuki", "kyun", "kyunki", "lagta", "lagt", "lagti", "le", "lekar", "lekin",
    "lest", "li", "liya", "liy", "lo", "log", "logon", "lol", "ltd", "lunga", "maan",
    "maana", "maan", "maani", "maano", "magar", "mai", "main", "main", "mainli", "mana",
    "mane", "mani", "mano", "mani", "mat", "mein", "mera", "mere", "mere", "meri",
    "mil", "mjhe", "moreov", "mostli", "much", "mujh", "mustnt","n","na", "naa", "naah",
    "nahi", "nahin", "nai", "nd", "ne", "nearli", "neech", "neednt", "nhi", "non",
    "noon", "noth", "p","par", "pata", "pe", "pehla", "pehl", "pehli", "peopl", "per",
    "phla", "phle", "phli", "plu","plz", "poora", "poori", "pura", "puri", "que","r","raha",
    "rahaa", "rahe", "rahi", "rakh", "rakha", "rakh", "rakhen", "rakhi", "rakho", "realli",
    "reht", "rha", "rhaa", "rhe", "rhi", "ri", "sa", "saara", "saar", "saath", "sab",
    "sabhi", "sabs", "sahi", "sakta", "saktaa", "sakt", "sakti", "sang", "sara", "sath",
    "secondli", "selv", "shant", "si", "sinc", "soch", "sub", "sup", "sure", "tab",
    "tabh", "tak", "tarah", "teeno", "teesra", "teesr", "teesri", "tend", "tera", "tere",
    "teri","theek", "thi", "tho", "thoda", "thodi", "thru", "thu", "tjhe", "toh", "tri",
    "tri", "truli", "tri", "tu", "tujh", "tum", "tumhara", "tumhar", "tumhari", "tune",
     "u","um", "umm", "un", "unh", "unhi", "unho", "unhon", "unka", "unkaa", "unk", "unki",
    "unko", "unn", "uns", "upar", "us", "usi", "use", "uska", "usk", "usn", "uss",
    "uss", "ussi", "vaala", "vaal", "vaali", "vahaan", "vahan", "vahi", "vahin", "vaisa",
    "vais", "vaisi", "vala", "vale", "vali", "via", "viz", "vo", "waala", "waal", "waali",
    "wagaira", "wagairah", "wagerah", "waha", "wahaan", "wahan", "wahi", "wahin", "waisa",
    "wais", "waisi", "wala", "wale", "wali", "wo", "woh", "wohi", "ya", "yadi", "yah",
    "yaha", "yahaan", "yahan", "yahi", "yahin", "ye", "yeah", "yeh", "yehi", "ye", "yup",
]

In [28]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')

stem=SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9\']+')
stops=set(stopwords.words('english'))
hindi_stops=set(custom_hindi_stops)

def preprocess(text):
  wordlist=[]

  for word in tokenizer.tokenize(text.lower()):
      if word not in stops and word not in custom_hindi_stops:
        stemmed_word = stem.stem(word)
        wordlist.append(stemmed_word)
  return wordlist

new_df['commentText']=new_df['commentText'].apply(preprocess)

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading words: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [29]:
new_df.shape

(9800, 2)

In [30]:
new_df.iloc[550:600]

,commentText,Labels
550,[superb],3
551,[ok],6
552,"[mam, tamatu, nibbu, yuz, krte]",6
553,[good],3
554,"[hi, sour, curd, food, curd, normal, sweet, cu...",7
555,[yummi],2
556,"[madam, mujh, aapki, biryani, bnane, trick, ac...",6
557,"[tri, process, today, delici, bt, rice, becom,...",7
558,"[thing, make, recip, look, easi]",4
559,"[ma'am, ora, water, pleas, repli, ma'am]",7


In [31]:
X=new_df['commentText']
y=new_df['Labels']

In [32]:
X = [' '.join(document) for document in X]

In [33]:
lbl=LabelEncoder()
y=lbl.fit_transform(y)

In [34]:
y=to_categorical(y,num_classes=7)

In [35]:
model_en=fasttext.load_model('./cc.en.300.bin')

In [36]:
X=np.array([model_en.get_sentence_vector(sentence) for sentence in X])
X

array([[-0.04865845, -0.02029513,  0.00281682, ...,  0.04467136,
         0.0015281 , -0.00535533],
       [ 0.00939113, -0.07598583, -0.01752039, ...,  0.07207707,
         0.03232015, -0.03124447],
       [ 0.02100777, -0.01061132,  0.00563888, ...,  0.06104375,
         0.01299675, -0.00439147],
       ...,
       [ 0.03667046, -0.00809222, -0.023444  , ...,  0.05898034,
         0.01075574, -0.0243484 ],
       [ 0.01814937, -0.00052348, -0.05576143, ...,  0.06553596,
         0.03396218, -0.02669046],
       [ 0.02075222, -0.06449727, -0.03291111, ...,  0.09440843,
         0.01172403, -0.03573036]], dtype=float32)

In [37]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.20)

In [38]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, PReLU,LeakyReLU
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2

np.random.seed(42)


# Reshape input data for LSTM
X_train = X_train.reshape(-1,1,300)
X_test = X_test.reshape(-1,1,300)

model = Sequential()

model.add(LSTM(64, input_shape=(1,300)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(25,activation=PReLU(),kernel_regularizer=l2(0.001)))

model.add(Dense(40,activation=LeakyReLU(),kernel_regularizer=l2(0.001)))

model.add(Dense(30,activation='relu',kernel_regularizer=l2(0.001)))

model.add(Dense(35,activation='tanh',kernel_regularizer=l2(0.001)))

model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Train the model on this fold
model.fit(X_train, Y_train, epochs=150, batch_size=150, validation_data=(X_test, Y_test), callbacks=[early_stopping])

Epoch 1/150
53/53 [==============================] - 93s 118ms/step - loss: 1.7215 - accuracy: 0.3940 - val_loss: 1.9313 - val_accuracy: 0.2862
Epoch 2/150
53/53 [==============================] - 1s 15ms/step - loss: 1.2456 - accuracy: 0.5811 - val_loss: 1.9590 - val_accuracy: 0.1679
Epoch 3/150
53/53 [==============================] - 0s 7ms/step - loss: 1.0638 - accuracy: 0.6478 - val_loss: 1.9344 - val_accuracy: 0.1714
Epoch 4/150
53/53 [==============================] - 0s 7ms/step - loss: 0.9730 - accuracy: 0.6818 - val_loss: 1.9280 - val_accuracy: 0.1918
Epoch 5/150
53/53 [==============================] - 0s 8ms/step - loss: 0.9283 - accuracy: 0.6960 - val_loss: 1.7568 - val_accuracy: 0.2189
Epoch 6/150
53/53 [==============================] - 0s 7ms/step - loss: 0.8879 - accuracy: 0.7165 - val_loss: 1.7175 - val_accuracy: 0.3357
Epoch 7/150
53/53 [==============================] - 0s 6ms/step - loss: 0.8667 - accuracy: 0.7193 - val_loss: 1.4461 - val_accuracy: 0.4260
Epoch 8/1

In [39]:
accuracy = model.evaluate(X_train, Y_train)[1]
print("Accuracy on train data: {:.2f}%".format(accuracy * 100))

245/245 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.8310
Accuracy on train data: 83.10%


In [40]:
accuracy = model.evaluate(X_test, Y_test)[1]
print("Accuracy on test data: {:.2f}%".format(accuracy * 100))

62/62 [==============================] - 0s 4ms/step - loss: 0.7617 - accuracy: 0.7526
Accuracy on test data: 75.26%
